In [217]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator
from qiskit import transpile

import random
import numpy as np

In [246]:
class BitCommitment:
    """ 
        Alice wants to send a boolean message (two possible values, e.g. YES/NO or True/False),
        to Bob, which he can only interpret at a later time when Alice wills. This procedure ensures
        that Alice is comitted to the particular Boolean value she initially sent, while Bob 
        can confirm this once Alice provides him with further information. This script also includes
        the possibility to include a potential eavesdropper Eve, who attempts to intercept the signal
        between Alice and Bob, and countermeasurements Alice and Bob can take to detect any such 
        eavesdroppers. 
    """
    def __init__(self,
                 num_Qbits: int = 10,
                 bool_message: bool = True):
        
        self.n = num_Qbits
        self.bool_message = bool_message

    def alice_sends_message(self):
        """
            Alice wants to send a boolean message to Bob. If the bool message is True, she sends n photons (Qbits),
            each randomly selected in |0> and |1> states (vertically and horizontally polarized
            photons). If False, she first applies Hadamard transformations to each randomly 
            selected state above (\pm 45 degree diagonally polarized photons) and then sends them 
            to Bob. Alice notes which polarization state (Qbit type and gate type) of each photon.
            At a later time, Alice reveals her boolean message by telling Bob via an unsecure channel
            what gate type she used (i.e. if she applied Hadamard transformations or not).
        """
        n = self.n
        bool_message = self.bool_message

        qreg = QuantumRegister(n, name="qreg")
        qc = QuantumCircuit(qreg)

        alice_qbit_list = []
        # Alice randomly selects Qbits with bit values 0 or 1
        for qbit in range(n):
            qbit_type = random.randint(0,1)
            if qbit_type==1:
                qc.x(qreg[qbit])
                # Store the polarization state of the qbits as chosen by Alice
            alice_qbit_list.append(qbit_type)
        qc.barrier()
        # Alice applied Hadamard transformations to the states above if her boolean
        # messafe is NO/False
        if bool_message:
            alice_gate_type = 0
        else:
            for qbit in range(n):
                qc.h(qreg[qbit])
            alice_gate_type = 1
        
        self.qreg = qreg
        self.qc = qc
        self.alice_qbit_list = np.array(alice_qbit_list)
        self.alice_gate_type = alice_gate_type

        print(f"\nAlice sends the boolean message {bool_message} in the form of a sequence of photons")

    def eve_evedrops(self):
        """
            Before Alice's message can reach Bob, Eve interrupts the signal. Since Alice has not yet
            declared which gate type she used (i.e. if she applied Hadamard transformations or not 
            before sending the induvidual Qbits), best Eve can do is randomly applying Hadamard 
            transformations onto the Qbits before making her measurements. To decieve Bob and Alice 
            of her interception Eve randomly applied Hadamard transformations to the outgoing photons 
            (initialized in the states |0> and |1> depending on her own measurements). But the chances
            of her picking the correct gate type for each gate type are 1/2^n.
        """
        n = self.n
        qreg = self.qreg
        qc = self.qc

        qc.barrier()
        eve_qbit_list = []
        eve_gate_list = []
        # Eve randomly applies Hadamard transformations to the states she
        # intercepted between Alice and Bob
        for qbit in range(n):
            gate_type = random.randint(0,1)
            if gate_type == 1:
                qc.h(qreg[qbit])
            # store gates applied by Eve
            eve_gate_list.append(gate_type)
    
        # Measure states
        qc.measure_all()
        simulator = AerSimulator()
        compiled_curcuit = transpile(qc, simulator)
        job = simulator.run(compiled_curcuit, shots=1)
        counts = job.result().get_counts()

        # Store Eve's measurements
        measurement_str = list(counts.keys())[0]
        for qbit_type_str in reversed(measurement_str):
            eve_qbit_list.append(int(qbit_type_str))
        
        # Delete Alice's curcuit, since Eve has interrupted the Alice's message to Bob
        del self.qc
        # Define new curcuit which Eve sends to Bob
        qreg = QuantumRegister(n, name="qreg")
        qc = QuantumCircuit(qreg)

        # Set qbit type according to what Eve measured
        for qbit, qbit_type in enumerate(eve_qbit_list):
            if qbit_type==1:
                qc.x(qreg[qbit])
        qc.barrier()
        # Eve re-applies gates as above before sending to Bob
        for qbit, gate_type in enumerate(eve_gate_list):
            if gate_type==1:
                qc.h(qreg[qbit])

        self.qc = qc
        self.eve_counts = counts
        self.eve_gate_list = np.array(eve_gate_list)
        self.eve_qbit_list = np.array(eve_qbit_list)

        print("\nEve intercepted the message, don't tell Alice or Bob!")

    def bob_receives_message(self):
        """
            Bob receives the message from Alice (or potentially from Eve if it has been intercepted).
            Bob randomly applies Hadamard transformations to the incoming Qbits and thereafter makes 
            measurements. 
        """
        n = self.n
        qreg = self.qreg
        qc = self.qc

        qc.barrier()
        bob_qbit_list = []
        bob_gate_list = []
        # Bob randomly applies Hadamard transformations to the incoming states
        for qbit in range(n):
            gate_type = random.randint(0,1)
            if gate_type == 1:
                qc.h(qreg[qbit])
            # Store which gates Bob applied
            bob_gate_list.append(gate_type)

        # Measure states    
        qc.measure_all()
        simulator = AerSimulator()
        compiled_curcuit = transpile(qc, simulator)
        job = simulator.run(compiled_curcuit, shots=1)
        counts = job.result().get_counts()

        # Store Bob's measurements
        measurement_str = list(counts.keys())[0]
        for qbit_type_str in reversed(measurement_str):
            bob_qbit_list.append(int(qbit_type_str))

        self.qc = qc
        self.counts = counts
        self.bob_gate_list = np.array(bob_gate_list)
        self.bob_qbit_list = np.array(bob_qbit_list)

        print("\nBob received the message from Alice and saved his measurements")

    def look_for_eavesdropper(self):
        n = self.n
        alice_qbit_list = self.alice_qbit_list
        bob_qbit_list = self.bob_qbit_list

        alice_gate_list = np.ones(n) * self.alice_gate_type
        bob_gate_list = self.bob_gate_list
        alice_bob_common_gates = np.where(alice_gate_list==bob_gate_list, 1, 0)
        print("\nAlice publicly declares what her boolean message was and thereby which gate she applied. Bob can now interpret the message he received")

        alice_bob_qbit_check = np.array([1 if (alice_qbit_list[i] == bob_qbit_list[i] or alice_bob_common_gates[i] == 0) else 0 for i in range(n)])
        print("\nAlice and Bob compare their measurements to make sure that Eve was not evesdropping.")

        if 0 in alice_bob_qbit_check:
            print("Oh no! Eve was evesdroppong!")
        else:
            certainty = np.round((1-1/2**(sum(alice_bob_common_gates))) * 100, 1)
            bob_bool_message = True if bob_gate_list[0] == 0 else False
            print(f"\nNo evesdropper detected. {certainty}% certain")
            print(f"Bob confirms that Alice indeed sent the boolean message {bob_bool_message}")
            self.bob_bool_message = bob_bool_message
        
        self.alice_bob_qbit_check = alice_bob_qbit_check
        

bit_commitment = BitCommitment(num_Qbits=10)

bit_commitment.alice_sends_message()
# bit_commitment.eve_evedrops() # uncomment this line if Eve evesdrops
bit_commitment.bob_receives_message()
bit_commitment.look_for_eavesdropper()


Alice sends the boolean message True in the form of a sequence of photons

Bob received the message from Alice and saved his measurements

Alice publicly declares what her boolean message was and thereby which gate she applied. Bob can now interpret the message he received

Alice and Bob compare their measurements to make sure that Eve was not evesdropping.

No evesdropper detected. 99.2% certain
Bob confirms that Alice indeed sent the boolean message True
